# Perplexity playbook

<p>
Mal Minhas, v0.3<br>
15.01.24
</p>

## 1. Introduction

Perplexity is an intelligent search engine with a chatbot sytle interface that leverages popular AI models to offer comprehensive answers to user inquiries.  It is able to search the internet to provide real-time information via a friendly user experience that includes native mobile apps.  The tool is specifically designed to perform as a research companion providing links to underlying source articles unlike ChatGPT in default mode.  Perplexity comes with an API called [`pplx-api`](https://blog.perplexity.ai/blog/introducing-pplx-api) which was introduced in October 2023.  See [here](https://docs.perplexity.ai/docs/getting-started) for a guide on how to set up an API key and get started with `pplx-api`.  The following examples assume you have set up the API key in an environment variable called `PERPLEXITY_API_KEY`.

Supported models are [as follows](https://docs.perplexity.ai/docs/model-cards):
```
Model	                  Context Length       Model Type
codellama-34b-instruct	      16384          Chat Completion
llama-2-70b-chat               4096          Chat Completion
mistral-7b-instruct [2]	       4096 [1]      Chat Completion
mixtral-8x7b-instruct	       4096 [1]      Chat Completion
pplx-7b-chat	               8192          Chat Completion
pplx-70b-chat	               4096          Chat Completion
pplx-7b-online	               4096          Chat Completion
pplx-70b-online	               4096          Chat Completion

[1] We will be increasing the context length of mistral-7b-instruct to 32k tokens (see roadmap).
[2] This model refers to the v0.2 release of mistral-7b-instruct.
```

The only API function currently supported is on `chat_completions` documented [here](https://docs.perplexity.ai/reference/post_chat_completions).  The next section outlines how to use it.

## 2. Basic Usage

Let's start with a few basics including a utility function to render text for display in Jupyter:

In [1]:
import os
import requests
from IPython.display import Markdown, display

def renderCompletion(text: str):
    display(Markdown(text))
    
API_KEY = os.environ.get("PERPLEXITY_API_KEY")
DEFAULT_MODEL = "mistral-7b-instruct"
DEFAULT_TOKENS = 1024
CONCISE_DETAIL = "Be precise and concise."
MID_LEVEL_DETAIL = "Explain in a few sentences."
GREAT_DETAIL = "Explain in a lot of detail."
POLITE_DETAIL = "You are an artificial intelligence assistant and you need to engage in a helpful, detailed, polite conversation with a user."
DEFAULT_TEMP = 0.0

The following is a convenience function that wraps the Perplexity API allowing different models to be used and also differing levels of depth of response detail:

In [2]:
def getPerplexityResponse(question:str, model:str=DEFAULT_MODEL, direction:str=MID_LEVEL_DETAIL, max_tokens:int=DEFAULT_TOKENS, temperature:float=DEFAULT_TEMP) -> str:
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": False,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": f"{direction}"
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    response = requests.post(url, json=payload, headers=headers).json()
    return renderCompletion(response.get("choices")[0].get("message").get("content"))

Let's test the function on an example query using the default Mistral 7b model and then Llama2.  Note the differences in response timing:

In [3]:
%%time

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
getPerplexityResponse(prompt)

In Southampton, UK, if you have a broken washing machine, you should call a professional appliance repair technician. They have the necessary skills and expertise to diagnose and fix common washing machine issues. You can look up local appliance repair services online or check directories like Yell or Thomson Local. Alternatively, you could contact a local appliance repair company, such as Southampton Appliance Repairs or A1 Appliance Repairs, for assistance. Make sure to describe the problem accurately when you call to help them prepare for the repair visit

CPU times: user 24.1 ms, sys: 8.55 ms, total: 32.7 ms
Wall time: 1.46 s


In [4]:
%%time

getPerplexityResponse(prompt, model='llama-2-70b-chat')

If you have a broken washing machine in Southampton, UK, you can call a local appliance repair service or a professional washing machine technician to diagnose and fix the issue. You can search online for appliance repair services in your area or check online directories such as Yell or Checkatrade to find a reputable technician. Additionally, you can also contact the manufacturer's customer service number to see if they can provide assistance or recommend a repair service

CPU times: user 29.8 ms, sys: 13.5 ms, total: 43.2 ms
Wall time: 2.33 s


In [5]:
%%time

getPerplexityResponse(prompt, model='codellama-34b-instruct', direction=POLITE_DETAIL)

I'm so sorry to hear that your washing machine is broken! There are a few options you can consider to get it mended.

1. Contact the manufacturer: If you have purchased the washing machine from a reputable brand, you can contact their customer service department to inquire about repair options. They may be able to provide you with information on authorized repair centers or repair kits that can help you fix the machine yourself.
2. Look for local appliance repair services: There may be local appliance repair services in your area that specialize in fixing washing machines. You can search online or check with local plumbers or electricians to find a reputable repair service.
3. Check with your insurance provider: If you have home insurance, you may be able to file a claim for the repair costs. Contact your insurance provider to see if they cover appliance repairs and what the process is for filing a claim.
4. Consider purchasing a new washing machine: If the repair costs are too high or you are unable to find a reliable repair service, it may be more cost-effective to purchase a new washing machine.

In terms of the trade you need to call, it depends on the type of repair you need. If the repair is minor and you can fix it yourself, you may not need to call any trade. However, if the repair requires more extensive work, you may need to call a plumber or an appliance repair technician.

I hope this information helps! If you have any further questions, please don't hesitate to ask

CPU times: user 23.2 ms, sys: 12.8 ms, total: 36 ms
Wall time: 4.57 s


Let's see how it handles a query with a more detailed response:

In [6]:
%%time

prompt = """"I have a broken garden gate which is no longer closing properly.  
What trade do I need to call to fix it?"""
getPerplexityResponse(prompt, direction=GREAT_DETAIL)

A broken garden gate that is no longer closing properly is an issue that requires the expertise of a tradesperson specializing in carpentry, specifically in the area of door repair and installation. Here's a detailed explanation of why this is the case:

A garden gate is a type of door that provides access to the outdoor space surrounding a property. It is typically made of wood, metal, or a combination of both, and it operates using hinges that attach it to the fence or wall. Over time, due to various factors such as weather conditions, wear and tear, or accidental damage, a garden gate may develop issues that prevent it from closing properly.

When a garden gate is not closing correctly, it can be caused by several different problems. For instance, the hinges may have become loose or rusted, the latch or locking mechanism may be faulty, or the frame of the gate may have warped or twisted. In some cases, the problem may be more complex, such as damage to the gatepost or the fence to which the gate is attached.

To repair a broken garden gate, a tradesperson with the necessary skills and expertise is required. This tradesperson is typically referred to as a carpenter or a door repair specialist. Carpenters are trained in the art of woodworking and are skilled in the design, construction, and repair of wooden structures, including doors and gates. Door repair specialists, on the other hand, focus specifically on the repair and installation of doors and their associated hardware.

The process of repairing a broken garden gate involves several steps. First, the carpenter or door repair specialist will inspect the gate to determine the cause of the problem. They will check the hinges, the latch or locking mechanism, and the frame of the gate for any signs of damage or wear. They may also check the gatepost and the fence to which the gate is attached to ensure that they are in good condition.

Once the cause of the problem has been identified, the carpenter or door repair specialist will then proceed to repair or replace the damaged components. For instance, if the hinges are loose, they may be tightened or replaced. If the latch or locking mechanism is faulty, it may be repaired or replaced. If the frame of the gate has warped or twisted, it may be straightened or replaced.

In some cases, the damage to the garden gate may be too extensive to be repaired, and a new gate may need to be installed. In this scenario, the carpenter or door repair specialist will measure the opening where the gate is to be installed and provide the customer with options for a new gate that fits the space and meets their budget and aesthetic preferences.

In summary, if you have a broken garden gate that is no longer closing properly, you should call a carpenter or a door repair specialist to fix it. These tradespeople have the necessary skills and expertise to diagnose and repair the problem, ensuring that your garden gate functions correctly once again

CPU times: user 30.9 ms, sys: 11 ms, total: 41.9 ms
Wall time: 5.9 s


## 3. Streaming interface

tbd

In [7]:
%%time

def getPerplexityStreamingResponse(question, model=DEFAULT_MODEL, direction=MID_LEVEL_DETAIL, max_tokens=DEFAULT_TOKENS, temperature=DEFAULT_TEMP):
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": True,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": direction,
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    print('start')
    response_stream = requests.post(url, json=payload, headers=headers)
    print('end')
    for response in response_stream:
        print(len(response))
    #return response.get("choices")[0].get("message").get("content")

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
#getPerplexityStreamingResponse(prompt, direction=GREAT_DETAIL)

CPU times: user 7 µs, sys: 6 µs, total: 13 µs
Wall time: 13.8 µs
